Generación de texto usando una RNN
===

* *30 min* | Última modificación: Abril 6, 2020.

Basado en https://www.tensorflow.org/tutorials/text/text_classification_rnn

## Importación de librerías

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

## Descarga del dataset y lectura de los datos

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


## Procesamiento del texto

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [15]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Construcción del modelo

In [16]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## Uso del modelo

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([29, 12, 22, 51,  1, 39, 50, 47, 56, 23, 49, 39, 23,  6, 49,  3, 15,
       25, 19,  1, 50, 48, 51,  0,  9,  0, 59, 24, 24, 63, 59, 17, 60, 20,
       30, 37, 28, 31, 39, 15, 27, 57,  0, 33, 20, 28, 39, 44, 53,  9, 19,
       10,  4, 54, 44, 30, 19, 24, 31, 64, 49, 60, 60, 19, 35, 37, 31, 46,
       53, 50, 60,  3, 52, 29, 35, 56, 57, 59, 16, 51, 14,  2, 42, 33,  6,
        7, 13,  7, 32,  1, 42, 61, 35, 47, 25, 17, 55, 41, 21, 59])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "\n\nHORTENSIO:\nSir, let me be so bold as ask you,\nDid you yet ever see Baptista's daughter?\n\nTRANIO:\nN"

Next Char Predictions: 
 'Q?Jm alirKkaK,k$CMG ljm\n3\nuLLyuEvHRYPSaCOs\nUHPafo3G:&pfRGLSzkvvGWYSholv$nQWrsuDmB!dU,-A-T dwWiMEqcIu'


## Entrenamiento del modelo

In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.175127


In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
EPOCHS=10

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/10
172/172 [==============================] - 7s 40ms/step - loss: 2.6536
Epoch 2/10
172/172 [==============================] - 6s 34ms/step - loss: 1.9498
Epoch 3/10
172/172 [==============================] - 6s 34ms/step - loss: 1.6860
Epoch 4/10
172/172 [==============================] - 6s 35ms/step - loss: 1.5411
Epoch 5/10
172/172 [==============================] - 6s 34ms/step - loss: 1.4541
Epoch 6/10
172/172 [==============================] - 6s 34ms/step - loss: 1.3952
Epoch 7/10
172/172 [==============================] - 6s 34ms/step - loss: 1.3497
Epoch 8/10
172/172 [==============================] - 6s 34ms/step - loss: 1.3097
Epoch 9/10
172/172 [==============================] - 6s 34ms/step - loss: 1.2764
Epoch 10/10
172/172 [==============================] - 6s 35ms/step - loss: 1.2435


## Generación de texto

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: Hastings with
Your two make gross my horses did umacten, or taumbted
You af their youth; we must here prick?

LUCENTIO:
Alas! I have heard his
cales in beasts to his heart.

HENRY BOLINGBROKE:
In entertain in marth and well-more than e.

ROMEO:
O, mute awhile; but Montague with his
as the world green sensied treasures of their
winded gone, but by both excel
With sughts and vowbrave youildaress proclaim prove.

Gallenderous
Let more than heaven? point, though good neire of mine,
Who wounds the act ordenly.

TRANIO:
In all quit isle, being rude awrieve to fear my heart,
Yet need. God-den claim of Menusos; how such a
man the second Kingsoo,
Look'd how concludents. Proceed those earthly in the
moour for all twice; if you part;
And Waling helm, hastily have the mild,
Seeming twites that got the child-but he that?
Hook! O thou with two
sheet particular sword; whose careguard not your
son, each one that is a woman of my foot!

Furse:
I never say 'gaiding and My brits no hare, she are t

## Entrenamiento personalizado

In [34]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [35]:
optimizer = tf.keras.optimizers.Adam()

In [36]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [37]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.17379903793335
Epoch 1 Batch 100 Loss 2.3642020225524902
Epoch 1 Loss 2.1451
Time taken for 1 epoch 6.280483961105347 sec

Epoch 2 Batch 0 Loss 2.144087791442871
Epoch 2 Batch 100 Loss 1.9279800653457642
Epoch 2 Loss 1.7839
Time taken for 1 epoch 5.3385679721832275 sec

Epoch 3 Batch 0 Loss 1.797566533088684
Epoch 3 Batch 100 Loss 1.7153781652450562
Epoch 3 Loss 1.6565
Time taken for 1 epoch 5.347963571548462 sec

Epoch 4 Batch 0 Loss 1.5770742893218994
Epoch 4 Batch 100 Loss 1.5482399463653564
Epoch 4 Loss 1.5179
Time taken for 1 epoch 5.357679128646851 sec

Epoch 5 Batch 0 Loss 1.506169080734253
Epoch 5 Batch 100 Loss 1.4482747316360474
Epoch 5 Loss 1.4387
Time taken for 1 epoch 5.294915437698364 sec

Epoch 6 Batch 0 Loss 1.3886843919754028
Epoch 6 Batch 100 Loss 1.3790349960327148
Epoch 6 Loss 1.3743
Time taken for 1 epoch 5.332662343978882 sec

Epoch 7 Batch 0 Loss 1.3449599742889404
Epoch 7 Batch 100 Loss 1.3802368640899658
Epoch 7 Loss 1.3486
Time taken for